# Importing Necesary Libraries

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        import numpy as np 
import pandas as pd 
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import cv2
import time

import warnings
warnings.filterwarnings("ignore")



from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import image_dataset_from_directory
from keras.utils.np_utils import to_categorical
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Rescaling
from tensorflow.keras.optimizers import Adam
import os

from time import time
from tensorflow.keras import utils


## GPU

In [ ]:
import tensorflow as tf
import scipy
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Data

In [ ]:
train_dir = '../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'
test_dir = '../input/asl-alphabet/asl_alphabet_test/asl_alphabet_test'
classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
           'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
           'W', 'X', 'Y', 'Z', 'nothing', 'space', 'del']
plt.figure(figsize=(11, 11))
for i in range (0,29):
    plt.subplot(7,7,i+1)
    plt.xticks([])
    plt.yticks([])
    path = train_dir + "/{0}/{0}1.jpg".format(classes[i])
    img = plt.imread(path)
    plt.imshow(img)
    plt.xlabel(classes[i])

# Loading Data

In [ ]:
import random
seed = 42  # Başlangıçta kullanılacak rastgele tohum değeri
random.seed(seed)

In [ ]:
!pip install split-folders

import splitfolders
splitfolders.ratio("/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/","/kaggle/working/output",ratio=(0.8,0.2))

In [ ]:
train_path= "/kaggle/working/output/train"
val_path = "/kaggle/working/output/val"

In [ ]:
img_size=(128,128)
batch=256
epoch=40
learning_rate = 0.001
callback= keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=8,
                                            restore_best_weights=True)

In [ ]:
from glob import glob
className=glob(train_path+"/*")
numberOfClass=len(className)
print("number of class:",numberOfClass)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=img_size,
    batch_size=batch,
    class_mode="categorical",
    shuffle=True,
    seed=seed,
)


val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=img_size,
    batch_size=batch,
    class_mode="categorical",
    shuffle=False,
    seed=seed,
)

# CNN Model

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same', input_shape=(128,128, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(numberOfClass, activation='softmax'))


adam=Adam(lr=learning_rate)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

# Train CNN Model

In [ ]:
hist=model.fit(train_generator,
               batch_size=batch,
               epochs=epoch,
               validation_data=val_generator,
               callbacks=callback)


# Model Evaluation

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()